# BIENVENIDO!

<br />

## PROCESAMIENTO DE CSV PARA COMENTARIOS DE FACEBOOK e INSTAGRAM

## Clasificación de sentimientos y emociones

**Por favor, el nombre del archivo ".csv" que esté escrito todo en minúsculas y que no use "_"**

**Es para evitar conflictos con la interface lógica**

**El usuario debe configurar variables en la 2da Celda**

**Ingrese el nombre del archivo y el valor True en la red social correspondiente**

**Asegurese de que haya un solo filtro de red social activado poniendo False en el otro valor**

<br />
<br />

**Comandos Jupyter Notebook:**<br />

<br />
<center><b>

```
shift + enter: ejecutar celda y pasar a la celda siguiente

ctrl + enter: ejecutar celda y quedarse en la celda correspondiente
```

</b></center>
<br />
<br />
<br />
<br />

In [66]:
import pickle
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import torch


# APP Reputación Digital
from app.m1_sentiment import M1_sentiment
from app.m2_emotions import M2_emotions
from app.m3_emotions import M3_emotions

# Librerías Reputación Digital
from tools.tokenizer import preprocesamiento, procesamiento
from tools.extraccion_muestras import nombres_propios
from tools.facebook import clean_facebook_comments
from tools.intagram import clean_instagram_comments

**Configurar archivo de lectura y filtro correspondiente:**

In [67]:
##  INPUT USUARIO  ##  INPUT USUARIO  ##  ##  INPUT USUARIO  ##  INPUT USUARIO  ##

archivo_lectura="sheet-diferentes-usos-sofi-comentario.csv"

##  INPUT USUARIO  ##  INPUT USUARIO  ##  ##  INPUT USUARIO  ##  INPUT USUARIO  ##



##  SELECCIONAR LA RED SOCIAL EN CUESTION  ##  SELECCIONAR LA RED SOCIAL EN CUESTION  ##
facebook = True
instagram = False
##  SELECCIONAR LA RED SOCIAL EN CUESTION  ##  SELECCIONAR LA RED SOCIAL EN CUESTION  ##


if sum([facebook, instagram]) != 1:
    print("Elija un solo filtro")
    exit()

In [68]:
df = pd.read_csv(archivo_lectura) 

if facebook:
    batch = clean_facebook_comments(df.content.to_list())
elif instagram:
    batch = clean_instagram_comments(df.content.to_list())
else:
    print("Error: no se selecciono ningún filtro de comentario")


i_nombre_propios = nombres_propios(batch)
batch = procesamiento(batch)

FileNotFoundError: [Errno 2] No such file or directory: 'sheet-diferentes-usos-sofi-comentario.csv'

In [ ]:
# CARGAR DATASET DE PROCESAMIENTO
df = pd.DataFrame({"content": batch})
df

In [ ]:
# PREDICCION DE SENTIMIENTO
df = M1_sentiment.main_df(df)

In [ ]:
# PREDICCION DE LAS EMOCIONES I

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL = "02shanky/finetuned-twitter-xlm-roberta-base-emotion"
tokenizer_i = AutoTokenizer.from_pretrained(MODEL)
model_i = AutoModelForSequenceClassification.from_pretrained(MODEL)

pipeline_i = pipeline('text-classification', model=model_i, tokenizer=tokenizer_i, device=device, top_k=None)

# Realizar predicción y ajustar resultados
predictions = M2_emotions.classify_tweets(pipeline_i, df, target="content")
predictions = M2_emotions.emotions_features(predictions)

# update dataset
df = pd.concat([df, predictions], axis=1)

In [ ]:
# CLASIFICACION DE EMOCIONES II
"""
Acá vale una aclaración con respecto a los scores.
Por lo general, un modelo predice etiquetas, y 
le otorga un score entre 0 y 1 a cada etiqueta.

Este modelo no, lo que hace es generar etiquetas,
a cada etiqueta le otorga un score, y la suma de
todos los scores es igual a 1.
"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_NAME = "joeddav/distilbert-base-uncased-go-emotions-student"
model_ii = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
tokenizer_ii = AutoTokenizer.from_pretrained(MODEL_NAME)

pipeline_ii = TextClassificationPipeline(model=model_ii, tokenizer=tokenizer_ii, device=device, top_k=None)

predictions = M3_emotions.classify_tweets(pipeline_ii, df, target="content")
predictions = M3_emotions.emotions_features(predictions)

df = pd.concat([df, predictions], axis=1)

In [ ]:
# AGREGANDO PALABRAS ESCRITAS LA PRIMERA LETRA EN MAYUSCULAS
try:
    df["palabras_titulo"] = i_nombre_propios[:len(i_nombre_propios)-len(i_nombre_propios)%4]

except ValueError:
    print("Error en la consistencia estructural de los datos.")
    # consultar con un pytólogo
df

In [ ]:
# DESCARGA
archivo_descarga= "RD-"+archivo_lectura
df.to_csv(archivo_descarga, index=False)

In [ ]:
# DESCARGA DIVIDIDA POR SENTIMIENTO

mask_negativo = (df.sentiment_i == "negative").values
mask_positivo = (df.sentiment_i == "positive").values
mask_neutro = (df.sentiment_i == "neutral").values

df_sentiment_positive= df.loc[mask_positivo]
df_sentiment_negative= df.loc[mask_negativo]
df_sentiment_neutral = df.loc[mask_neutro]

## Conteo de las predicciones
print(f"Predicciones {archivo_lectura}:")
print("Negativos: ", mask_negativo.sum())
print("Positivos: ", mask_positivo.sum())
print("Neutro: ", mask_neutro.sum())


In [ ]:
df_sentiment_negative.to_csv(f"RD_negative-{archivo_lectura}")
df_sentiment_positive.to_csv(f"RD_positive-{archivo_lectura}")
df_sentiment_neutral.to_csv(f"RD_neutral-{archivo_lectura}")

In [ ]:
print("Programa finalizado de forma exitosa")